# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
## Load Cities.csv as DataFrame
cities_df = pd.read_csv('../WeatherPy/Output/Cities.csv')
## Display DataFrame preview
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,42.80,75,0,6.91,CL,1619720720
1,Ténenkou,14.4572,-4.9169,103.69,17,76,13.04,ML,1619720952
2,Lamar,33.6668,-95.5836,73.40,78,90,6.96,US,1619720952
3,Torbat-e Jām,35.2440,60.6225,73.08,39,93,14.85,IR,1619720953
4,Busselton,-33.6500,115.3333,50.00,98,86,14.18,AU,1619720747


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
## Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
locations = cities_df[['Lat', 'Lng']].values.tolist()
type(locations[0][0])

float

In [5]:
max_humidity = cities_df['Humidity'].max()
max_humidity

100

In [6]:
fig = gmaps.figure(center=(15,15), zoom_level=2.1)

locations = cities_df[['Lat', 'Lng']].values.tolist()
weights = cities_df['Humidity'].values.tolist()
max_humidity = cities_df['Humidity'].max()

heat_layer = gmaps.heatmap_layer(
    locations=locations,
    weights=weights,
    dissipating=False,
    max_intensity=max_humidity,
    point_radius = 3
)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
len(cities_df)

562

In [8]:
ideal_conditions = (cities_df['Max Temp'] > 70) & \
                   (cities_df['Max Temp'] < 80) & \
                   (cities_df['Wind Speed'] < 10) & \
                   (cities_df['Cloudiness'] == 0)

In [9]:
sum(ideal_conditions)

16

In [10]:
ideal_cities_df = cities_df[['City', 'Country', 'Lat', 'Lng']].loc[ideal_conditions]
ideal_cities_df.reset_index(drop=True, inplace=True)
ideal_cities_df.dropna(axis=0, inplace=True)

In [11]:
ideal_cities_df

,City,Country,Lat,Lng
0,Lompoc,US,34.6391,-120.4579
1,Karratha,AU,-20.7377,116.8463
2,Chui,UY,-33.6971,-53.4616
3,Inhambane,MZ,-23.8650,35.3833
4,Guangzhou,CN,23.1167,113.2500
5,Kiryat Gat,IL,31.6100,34.7642
6,Saint-Philippe,RE,-21.3585,55.7679
7,La Plata,AR,-34.9215,-57.9545
8,Rocha,UY,-34.4833,-54.3333
9,San Lorenzo,PY,-25.3333,-57.5333


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
## GOOGLE MAPS API

## Nearby Search URL
nearby_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

## Nearby Search Parameters
params = {
    'key': g_key,
    'radius': 5000,
    'type': "lodging",
    'language': "en"
}

In [13]:
hotel_name = []
hotel_lat = []
hotel_lng = []

## Print Log Header
print("Beginning Data Retrieval")
print("------------------------------")

for index, row in ideal_cities_df.iterrows():
    params['location'] = f"{row['Lat']},{row['Lng']}"
    response = requests.get(nearby_url, params=params).json()
    first_result = response['results'][0]
    
    print(f"Processing {row['City']}:")
    
    try:
        print(f" - First hotel is {first_result['name']}")
        hotel_name.append(first_result['name'])
        hotel_lat.append(first_result['geometry']['location']['lat'])
        hotel_lng.append(first_result['geometry']['location']['lng'])

    except (KeyError, IndexError):
        print("- Missing field/result... skipping.")
        
## Print Log Footer
print("------------------------------")        
print("Data Retrieval Complete")
print("------------------------------")

Beginning Data Retrieval
------------------------------
Processing Lompoc:
 - First hotel is Holiday Inn Express Lompoc
Processing Karratha:
 - First hotel is ibis Styles Karratha
Processing Chui:
 - First hotel is Nuevo Hotel Plaza
Processing Inhambane:
 - First hotel is Oceano
Processing Guangzhou:
 - First hotel is Guangzhou Hotel
Processing Kiryat Gat:
 - First hotel is OlusHome
Processing Saint-Philippe:
 - First hotel is Chambres d'hôte "La Trinité"
Processing La Plata:
 - First hotel is Hotel Corregidor
Processing Rocha:
 - First hotel is Beleza Pura B&B
Processing San Lorenzo:
 - First hotel is Hotel del Sur
Processing Ushtobe:
 - First hotel is Gostinitsa "Diyar"
Processing Saint-Joseph:
 - First hotel is "Plantation Bed and Breakfast
Processing Chiredzi:
 - First hotel is The Nesbitt Arms
Processing Maxixe:
 - First hotel is Oceano
Processing Neuquén:
 - First hotel is Hotel del Comahue
Processing Dom Pedrito:
 - First hotel is Hotel Alexandre
------------------------------
D

In [14]:
hotel_df = ideal_cities_df[['City', 'Country']]
hotel_df.insert(0, 'Hotel Name', hotel_name)
hotel_df.insert(len(hotel_df.columns), 'Lat', hotel_lat)
hotel_df.insert(len(hotel_df.columns), 'Lng', hotel_lng)

hotel_df

,Hotel Name,City,Country,Lat,Lng
0,Holiday Inn Express Lompoc,Lompoc,US,34.662502,-120.458466
1,ibis Styles Karratha,Karratha,AU,-20.732225,116.846744
2,Nuevo Hotel Plaza,Chui,UY,-33.694177,-53.455633
3,Oceano,Inhambane,MZ,-23.861426,35.346690
4,Guangzhou Hotel,Guangzhou,CN,23.115819,113.265594
5,OlusHome,Kiryat Gat,IL,31.613093,34.716021
6,"Chambres d'hôte ""La Trinité""",Saint-Philippe,RE,-21.358434,55.770322
7,Hotel Corregidor,La Plata,AR,-34.915296,-57.946396
8,Beleza Pura B&B,Rocha,UY,-34.490105,-54.322433
9,Hotel del Sur,San Lorenzo,PY,-25.335532,-57.531631


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))